In [1]:
import sys
import numpy as np
from matplotlib import pyplot as plt
sys.path.append('..')
from scipy.constants import h, hbar, e

Define a circuit for a flux-tunable transmon qubit with $E_C=0.3, E_{J1}=7 ~\text{and}~ E_{J2}=15$ GHz. Also we added a small inductance $m$ represended a coil element for flux tunability. TLSim2 works with variables in SI system, therefore we need to convert all parameters.

In [2]:
from tlsim2.lumped import *
from tlsim2.circuit import Circuit
sys_big = Circuit()

EC = 0.3
EJ1 = 7
EJ2 = 15


C = e ** 2 / (EC * 1e9 * h) / 2 # in F
EJ1 = EJ1 * 1e9 * h # in J
EJ2 = EJ2 * 1e9 * h
m = 10e-14 # in H

cap_q = Capacitor(name='C_q', c=C)
jj1 = JosephsonJunction(name='JJ1', ej=EJ1)
jj2 = JosephsonJunction(name='JJ2', ej=EJ2)
coil = Inductor(name='M', l=m)

sys_big.add_element(cap_q, {'i': 1, 'o': 0})
sys_big.add_element(jj1, {'i': 1, 'o': 0})
sys_big.add_element(jj2, {'i': 1, 'o': 2})
sys_big.add_element(coil, {'i': 0, 'o': 2})

sys_big.short(0)

In the linear approximation, we replace all Josepshon junctions with inductances. In this approximation we can obtain inverse inductance, capacitance and inverse resistance matrices. Also here we define node basis $\vec{\varphi} = (\varphi_1, \varphi_2, \ldots) ^T$. In the linear approximation we can find eigenmodes of the system using dynamical equations.

In [3]:
li, c, ri, node_names = sys_big.get_system_licri()
print('Node names in the system are ', node_names)

w, v, node_names = sys_big.compute_system_modes()
frequencies = np.imag(w/(2*np.pi)/1e9)
frequencies = frequencies[frequencies > 0]
print(r'E01 in the linear approximation = %.3f GHz' % frequencies[0])

Node names in the system are  [1, 2]
E01 in the linear approximation = 7.266 GHz


D:\tlsim2\example_notebooks\..\tlsim2\circuit.py:149: RuntimeWarning: invalid value encountered in multiply
  self.w = w * np.sqrt(max_li / max_c)
D:\tlsim2\example_notebooks\..\tlsim2\circuit.py:153: RuntimeWarning: invalid value encountered in multiply
  return w * np.sqrt(max_li / max_c), v, node_names
C:\Users\alena\AppData\Local\Temp\ipykernel_14472\2971142407.py:5: RuntimeWarning: invalid value encountered in true_divide
  frequencies = np.imag(w/(2*np.pi)/1e9)


For quntization we need to define all variables $\vec{\theta}$ presented in the system. According to the Koch article we can classify all variables to frozen, free, periodic and extended. We use the Gauss-Jordan elimination method to find a fundamental system of solutions.

For frozen variables we make zero flux difference across all capacitors.
For free variables we make zero flux difference across all elements where dc current flow is posiible.
For periodic variables we make zero flux difference across all elements which do not form superconducting islands where charge tunneling is possible.

In [4]:
basis, fss_types = sys_big.get_variable_transformation()
print('Linear transformation matrix is \n {} with variables {}'.format(basis, fss_types))

Linear transformation matrix is 
 [[-0.  1.]
 [ 1.  0.]] with variables ['frozen', 'periodic']


For further quantization we eliminate frozen $\theta_{f}$ and free $\theta_{c}$ variables. Frozen variables are corresponding to massless particles with potential energy. We eliminate these variables to avoid any problems with the Legendre transformation. Also we can eliminate free variables, because for theese variables the conjugate charge $Q_c$ is a conserved quantity. In other words, this type of variables is formed by islands where charge tunneling is not possible. Also it can be noted that for free variables the energy spectrum is not discrete (infinite movement).

After all these manipulation we create a qunatum circuit and define quantum variables.

In [5]:
sys_big.create_quantum_circuit()
sys_big.variables

for var in sys_big.variables:
    print(var.name)

θ_0_periodic


The next step is calculate charge and phase potential using a grid for created variables.

In [6]:
sys_big.calculate_potentials();

After that we can crete the Hamiltonian operator in phase representation and find discrete eigenenergies and wavefunctions.

In [7]:
operator = sys_big.hamiltonian
from scipy.sparse.linalg import eigs
energies, wavefunctions = eigs(operator, k=5, which='SR', maxiter=1000)

energ = np.real(energies - energies[0]) / h
print(r'E01 = %.3f GHz' % ((energ[1] - energ[0]) / 1e9))

E01 = 6.952 GHz


For transmon qubit we can use the following equantion from Koch article
$$
E_{10} = \sqrt{8 E_C E_J} - E_C
$$
This equation gives only approximated solution for the first order perturbation theory.

In [8]:
EC = e ** 2 / C / 2 / h / 1e9 # charge energy in GHz
EJ = (EJ1 + EJ2) / h / 1e9 # Josephson energy in GHz
E10 = np.sqrt(8 * EC * EJ) - EC
print(r'E01 = %.3f GHz' % E10)

E01 = 6.966 GHz
